In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

In [3]:
model = models.resnet18(pretrained=True)

c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
num_features = model.fc.in_features
num_features

512

In [49]:
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd

In [8]:
data_train = pd.read_csv("../data/data_train_f.csv")
data_train.head()

,path,label,conf,revalidated_manually
0,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1
1,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1
2,.\data\train\subject-1\fall\1_backward_falls\f...,1,0.444923,1
3,.\data\train\subject-1\fall\1_backward_falls\f...,1,0.616470,1
4,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1


In [28]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths 
        self.labels = labels
        self.transform = transform

        for i in range(len(self.image_paths)):
            self.image_paths[i] = "..\\" + self.image_paths[i]

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [57]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data_train, test_size=0.2, random_state=42)

In [58]:
train_data_paths = train_data['path'].values.tolist()
val_data_paths = val_data['path'].values.tolist()

train_data_labels = train_data['label'].values.tolist()
val_data_labels = val_data['label'].values.tolist()

train_dataset = CustomImageDataset(image_paths=train_data_paths, labels=train_data_labels, transform=transform)
val_dataset = CustomImageDataset(image_paths=val_data_paths, labels=val_data_labels, transform=transform)

In [59]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [60]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [53]:
for inputs, labels in train_loader:
    print(inputs.shape, labels.shape)
    outputs = model(inputs)
    loss = criterion(outputs, labels.unsqueeze(1).float())
    print(loss)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(0.1177, grad_fn=<BinaryCrossEntropyBackward0>)


In [62]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.unsqueeze(1).float()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total * 100
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}")

    if (epoch + 1) % 5 == 0:
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        test_acc = correct / total * 100
        print(f"Test Accuracy: {test_acc:.2f}%")


Epoch 1/10, Loss: 0.0450, Accuracy: 98.60
Epoch 2/10, Loss: 0.0161, Accuracy: 99.39
Epoch 3/10, Loss: 0.0319, Accuracy: 98.95
Epoch 4/10, Loss: 0.0065, Accuracy: 99.83
Epoch 5/10, Loss: 0.0215, Accuracy: 99.45
Test Accuracy: 898.37%
Epoch 6/10, Loss: 0.0142, Accuracy: 99.53
Epoch 7/10, Loss: 0.0091, Accuracy: 99.77
Epoch 8/10, Loss: 0.0040, Accuracy: 99.85
Epoch 9/10, Loss: 0.0041, Accuracy: 99.91
Epoch 10/10, Loss: 0.0029, Accuracy: 99.91
Test Accuracy: 897.67%


In [63]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss 
}, "../weights/checkpoint.pth")

In [64]:
torch.save(model.state_dict(), "../weights/weights_001.pth")